In [1]:
from sqlalchemy.engine import create_engine
from pandas.io import sql
import sqlite3
import pandas as pd

%load_ext sql
%sql sqlite:///crimen.db

global database
database = "crimen.db"

def result_query_run(query):
    connection = None
    result = None
    try:
        connection = sqlite3.connect(database)
        result = pd.read_sql_query(query, connection)
        connection.commit()
    except sqlite3.Error as e:
        print("Error executing the query:", e)
    finally:
        if connection is not None:
            connection.close()
    if result is None:
        return "No hay salida..."
    return result

def simple_query_run(query):
    connection = None
    try:
        connection = sqlite3.connect(database)
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
    except sqlite3.Error as e:
        print("Error executing the query:", e)
    finally:
        if connection is not None:
            connection.close()

In [12]:
# Description and type of crime, of the first 10 crimes of district 10301 (Desamparados, of Desamparados).
# Descripción y tipo de crimen, de los primeros 10 crímenes del distrito 10301 (Desamparados, de Desamparados).
print(result_query_run("SELECT crimen.tipo, crimen.descripcion FROM crimen WHERE crimen.id_distrito = 10301 LIMIT 10"))

                tipo               descripcion
0             asalto               arma blanca
1             asalto               arma blanca
2              hurto                carterista
3              hurto  ardid previo/distraccion
4  tacha de vehiculo         tacha de vehiculo
5             asalto               desconocido
6               robo                 forzadura
7  tacha de vehiculo         tacha de vehiculo
8             asalto             arma de fuego
9             asalto             arma de fuego


In [11]:
# Type of crime and description, of the first 15 crimes that occurred on Christmas 2022.
# Tipo de crimen y descripción, de los primeros 15 crímenes ocurridos en Navidad del 2022.
print(result_query_run("SELECT crimen.tipo, crimen.descripcion FROM crimen WHERE crimen.fecha = '2022-12-25' LIMIT 15"))

      tipo    descripcion
0     robo      forzadura
1     robo      forzadura
2     robo      forzadura
3     robo      forzadura
4     robo      forzadura
5     robo      forzadura
6     robo      forzadura
7     robo      forzadura
8     robo      forzadura
9     robo      forzadura
10    robo      forzadura
11   hurto     carterista
12   hurto     carterista
13   hurto     carterista
14  asalto  arma de fuego


In [21]:
# Identification and date of the crime, and nationality of the victim of all assaults occurred to non-Costa Ricans, of legal age, in Tamarindo, Guanacaste, during 2020.
# Identificación y fecha del crimen, y nacionalidad de la víctima de todos los asaltos ocurridos a no costarricenses, adultos mayores y mayores de edad, en Tamarindo, Guanacaste, durante el 2020.
print(result_query_run(
    """
    SELECT crimen.id_crimen, crimen.fecha, victima.nacionalidad 
    FROM crimen, victima, crimen_victima
    WHERE crimen.id_crimen = crimen_victima.id_crimen AND
    crimen_victima.id_victima = victima.id_victima AND
    crimen.tipo = 'asalto' AND
    crimen.id_distrito = 50309 AND
    crimen.fecha BETWEEN '2020-01-01' AND '2020-12-31' AND
    victima.nacionalidad != 'costa rica' AND
    (victima.grupo_etario = 'AAMM' OR 
    victima.grupo_etario = 'MAEE')
    """
))

   id_crimen       fecha    nacionalidad
0         87  2020-01-01   corea del sur
1       2079  2020-01-15       nicaragua
2      34855  2020-11-23  estados unidos
